In [3]:
import os
import requests
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [ ]:
@retry(wait=wait_random_exponential(min=20, max=50), stop=stop_after_attempt(200))
def summarize_dialogue(chat_history: list[dict], api_key: str):
    headers = {"Authorization": f"Bearer {api_key}"}
    url = "https://api.gpt.mws.ru/v1/chat/completions"
    model_name = 'deepseek-r1-distill-qwen-32b'
    system_prompt = """Ты - русскоязычный ассистент для суммаризации диалога.
История этого диалога - это история чата между другим ассистентом и оператором колл-центра.
Твоя задача - резюмировать всю информацию, которая есть у тебя в распоряжении.
Не отвечай на вопросы заного и не добавляй новую информацию
Отвечай на русском языке"""

    user_prompt = """Внимательно рассмотри историю и для кажтого ответа определи, 
с какими вопросами обращался оператор и удалось ли ассистенту выдать корректный ответ.
Выведи свою суммаризацию и в конце сделай вердикт по всему диалогу в одном предложении.
Отвечай на русском языке"""
    chat_history.extend([
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": user_prompt}
    ])
    data = {
        "model" : model_name,
        "messages" : chat_history,
        "temperature" : 0,
        "n" : 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        text = response.json()['choices'][0]['message']['content']
        think_end = text.index('</think>')
        start = think_end + len('</think>')
        return text[start:].strip()
    except Exception as e:
        print(f'{e}: error in summarize_dialogue function')
        return e

In [11]:
load_dotenv('.env.private', override=True)
api_key = os.getenv('API_KEY')

system_prompt = f"""Ты ассистент для нахождения ответа на вопрос пользователя
Твоя задача - внимательно проанализировать запрос пользователя и найти информацию в предоставленном тексте
Постарайся отвечать кратко"""
user_prompt = f"""Информация: Вы можете посмотреть доступные опции и подключить их в разделе Каталог > Услуги > Возможности при нуле Как проверить баланс и пакеты по тарифу\n\nТекст описывает, как проверить остаток интернет-пакета, дни до его обновления и условия тарифа на главном экране. Изменения условий оплаты или услуги могут произойти после бесплатного периода. Стоимость пакета списывается при подключении. Подробности тарифа, включая его название, размер пакетов и остатки, доступны\n\n Поэтому трафик по подписке может быть не виден сразу, но он точно есть Как это выглядит: в приложении Мой МТС в Личном кабинете В приложении и в Личном кабинете можно нажать на трафик, чтобы увидеть все подключённые пакеты\n\nразмер и остатки пакетов интернета, минут, СМС  Чтобы открыть полное описание своего тарифа:   1  На главной странице нажмите на его название   2  На открывшемся экране:\n\nВ разделе "Услуги" каталога можно подключить опции и проверить баланс. Нажмите на название тарифа для уточнения платы и даты списания. Кнопка "Настроить" доступна для тарифов с пакетами. Полное описание тарифа открывается при нажатии "О тарифе". Стоимость услуг и платные оп\n\nКак узнать подробности о своём тарифе 1  На главном экране вы можете узнать: название своего тарифа размер пакетов интернета, минут, СМС и их остатки\n\nПроверить подаренные минуты можно в Личном кабинете  В Мой МТС отображаются минуты, которые входят в ваш тариф Информация в виджетах С помощью виджетов приложения Мой МТС можно:  уточнить абонентскую плату и баланс"""
assistant_response = 'Для проверки баланса и пакетов по тарифу:\n\n1. В приложении "Мой МТС" или Личном кабинете нажмите на название тарифа на главной странице.\n2. На открывшемся экране можно увидеть:\n   - Название тарифа\n   - Размер и остатки пакетов интернета, минут, SMS\n   - Подаренные минуты (доступны в Личном кабинете)\n3. Для получения полной информации о тарифе нажмите "О тарифе".\n4. Абонентская плата и баланс можно проверить через виджеты приложения.'
chat_history = [
    #{"role":"system", "content":system_prompt},
    {"role":"user", "content": user_prompt},
    {"role":"assistant", "content": assistant_response}
]
answer = summarize_dialogue(chat_history, api_key)
answer

'### Суммаризация:\n1. **Вопрос оператора:** Как проверить баланс и пакеты по тарифу?  \n   **Ответ ассистента:** Указаны шаги для проверки через приложение "Мой МТС" и Личный кабинет, а также информация о том, как найти полное описание тарифа.  \n   **Корректность ответа:** Да, информация предоставлена полно и понятно.\n\n2. **Вопрос оператора:** Как подключить опции и проверить их баланс?  \n   **Ответ ассистента:** Указано, что опции можно подключить в разделе "Услуги" каталога, а также как проверить баланс через приложение.  \n   **Корректность ответа:** Да, информация предоставлена полно и понятно.\n\n### Вердикт:\nАссистент корректно ответил на все вопросы оператора, предоставив полную и понятную информацию о проверке баланса, пакетов и доступных опций.'

In [12]:
print(answer)

### Суммаризация:
1. **Вопрос оператора:** Как проверить баланс и пакеты по тарифу?  
   **Ответ ассистента:** Указаны шаги для проверки через приложение "Мой МТС" и Личный кабинет, а также информация о том, как найти полное описание тарифа.  
   **Корректность ответа:** Да, информация предоставлена полно и понятно.

2. **Вопрос оператора:** Как подключить опции и проверить их баланс?  
   **Ответ ассистента:** Указано, что опции можно подключить в разделе "Услуги" каталога, а также как проверить баланс через приложение.  
   **Корректность ответа:** Да, информация предоставлена полно и понятно.

### Вердикт:
Ассистент корректно ответил на все вопросы оператора, предоставив полную и понятную информацию о проверке баланса, пакетов и доступных опций.
